# 6.2 Layer Normalization (LN)
本节旨在介绍LN及其实现方法，并将其与BN进行对比，分析各自在不同模型结构中的优势和缺陷

## 6.2.1 Layer Normalization 原理
为解决BN在循环网络结构（如RNN）上应用不明确的问题，以及不能应用于在线学习任务的问题，2016年多伦多大学提出Layer Normalization(LN)，通过在单个样本上对整层layer进行标准化，而不是在某个layer对所有样本进行标准化,因而可以有效地应用在循环网络中。

即对于RNN而言，LN对第i个样本在第t个时刻（第t层layer）的所有隐藏特征做标准化，BN对第t个时刻（第t层layer）的第j个隐藏特征在batch所有样本上做标准化。

定义： Input values in all neurons in the same layer are normalized for each data sample.

<p align=center>
<img src="./fig/6-10.png" width=600>
</p>

<p align=center>
<img src="./fig/6-11.png" width=600>
</p>

+ 对于LN在通用的ANN当中的应用，可以看作是每一层layer的所有神经元特征在激活前做标准化，即：

<p align=center>
<img src="./fig/6-12.png" width=600>
</p>

+ 对于LN在循环网络结构RNN当中的应用，每一层layer $l$在这里对应为时间步$t$，可以看作是每个时间步对所有hidden variable在激活前做标准化，即：

<p align=center>
<img src="./fig/6-13.png" width=600>
</p>


## 6.2.2 LN的优势与缺陷
LN的优势：
+ 在循环神经网络结构中，使用LN比BN效果更优，可以更快更好地达到最优效果；
+ LN与batchsize无关，不需要保存mini-batch的均值与方差，节省了额外的存储空间；

LN的缺点：
+ 对于非序列型数据的处理能力不如BN

## 6.2.3 LN的Torch实现与应用
首先对torch中的nn.LayerNorm及其用法进行介绍，而后通过一个例子展现LN在网络设计中的应用。

### 6.2.3.1 nn.LayerNorm介绍
```
nn.LayerNorm(normalized_shape, eps = 1e-5, elementwise_affine = True)
```
`normalized_shape`：每次归一化的shape。比如如果输入为图片（N，C，H，W），则 `normalized_shape=[C, H, W]`

<p align=center>
<img src="./fig/6-14.png" width=700>
</p>

`eps`：分母修正项$\epsilon%

`elementwise_affine`：是否要对每个元素做affine transform

Torch文档给出的nn.LayerNorm的使用简例：

In [2]:
import torch
from torch import nn

# NLP Example
batch, sentence_length, embedding_dim = 20, 5, 10 # LN would be conducted to embeddings
embedding = torch.randn(batch, sentence_length, embedding_dim)
layer_norm = nn.LayerNorm(embedding_dim) # 20-batch, 5-numOfLayers
layer_norm(embedding)

# Image Example
N, C, H, W = 20, 5, 10, 10
input = torch.randn(N, C, H, W)
layer_norm = nn.LayerNorm([C, H, W]) # one Layer, C featuresMap, HxW pixes
output = layer_norm(input)

### 6.2.3.2 LN在网络设计中的应用
本节通过一个简单的LeNet，实现一种带有BN的CNN网络（LN通常放在激活函数 前或后，根据实验结果好坏来确定）。关于实验结果对比可以参考[Layer Normalization in Pytorch](https://wandb.ai/wandb_fc/LayerNorm/reports/Layer-Normalization-in-Pytorch-With-Examples---VmlldzoxMjk5MTk1)

In [3]:
import torch
from torch import nn

class LeNet(nn.Module):
    @staticmethod
    def calc_activation_shape(dim, kernelSize, diliation=(1,1), stride=(1,1), padding=(0,0)):
        def shape_each_dim(i):
            odim_i = dim[i] + 2*padding[i] - diliation[i] * (kernelSize[i]-1) - 1 # Common Equation
            return (odim_i / stride[i]) + 1
        
        return shape_each_dim(0), shape_each_dim(1)
    
    def __init__(self, idim):
        # idim: image dimension
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        ln_shape = LeNet.calc_activation_shape(idim, 5)
        self.BN1 = nn.LayerNorm([20, ln_shape]) # you can calculate by yourself, and fill in 'ln_shape'
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        ln_shape = LeNet.calc_activation_shape(ln_shape, 5)
        self.BN2 = nn.LayerNorm([50, ln_shape])
        self.Linear1 = nn.Linear(800, 500)
        self.Linear2 = nn.Linear(500, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU(self.BN1(x))
        x = self.conv2(x)
        x = nn.ReLU(self.BN2(x))
        x = torch.flatten(x, start_dim=1)
        x = self.Linear1(x)
        x = nn.ReLU(x)
        x = self.Linear2(x)
        return x

## 6.2.4 BN与LN的对比总结
[一张图清楚说明BN与LN的归一化方式](https://blog.csdn.net/u011447962/article/details/117714531?spm=1001.2101.3001.6650.2&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-2-117714531-blog-97028488.pc_relevant_paycolumn_v3&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-2-117714531-blog-97028488.pc_relevant_paycolumn_v3)，其中蓝色区域代表做同一次归一化的样本

<p align=center>
<img src="./fig/6-9.png" width=600>
</p>

在CNN这种（N,C,H,W）样本中，C即指代通道数，BN对每个通道的所有样本做归一化，LN对每个样本的所有通道做归一化。